# Análisis Univariante: Satisfacción

Este notebook realiza un análisis univariante exhaustivo de las métricas de satisfacción de estudiantes y profesores en la UPV.

**Objetivos:**
- Analizar la distribución de satisfacción de alumnos y profesores
- Identificar patrones, tendencias y anomalías
- Evaluar la calidad de los datos
- Proporcionar visualizaciones claras y estadísticas descriptivas

## 1. Librerías Requeridas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerías cargadas exitosamente")

## 2. Cargar y Explorar Datos de Satisfacción

In [ ]:
# Cargar el panel maestro
panel_maestro = pd.read_csv('../data_extraction/panel_maestro_UPV.csv', encoding='utf-8')

print("📊 Información General del Dataset:")
print(f"  • Dimensiones: {panel_maestro.shape[0]} filas × {panel_maestro.shape[1]} columnas")
print(f"  • Peso: {panel_maestro.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n📋 Columnas de Satisfacción:")
satisfaction_cols = ['satisfaccion_alumnos', 'satisfaccion_profesores', 'diferencia_satis', 'satisfaccion_promedio']
for col in satisfaction_cols:
    print(f"  • {col}")

print("\n🔍 Primeras filas del dataset:")
display(panel_maestro[['CURSO', 'TITULACION', 'CENTRO', 'año'] + satisfaction_cols].head(10))

In [ ]:
# Seleccionar solo las columnas de satisfacción
satisfaction_data = panel_maestro[satisfaction_cols].copy()

print("📊 Información de Tipos de Datos:")
print(satisfaction_data.dtypes)

print("\n📈 Información General:")
satisfaction_data.info()

## 3. Estadísticas Descriptivas

In [ ]:
print("📊 ESTADÍSTICAS DESCRIPTIVAS COMPLETAS\n")
print("="*100)

for col in satisfaction_cols:
    print(f"\n🔹 {col.upper()}")
    print("-" * 100)
    
    data = panel_maestro[col].dropna()
    
    print(f"  Observaciones válidas: {len(data)}/{len(panel_maestro)} ({100*len(data)/len(panel_maestro):.2f}%)")
    print(f"  Valores faltantes: {panel_maestro[col].isna().sum()} ({100*panel_maestro[col].isna().sum()/len(panel_maestro):.2f}%)")
    print(f"\n  Medidas de Tendencia Central:")
    print(f"    • Media: {data.mean():.4f}")
    print(f"    • Mediana: {data.median():.4f}")
    print(f"    • Moda: {data.mode().values[0] if len(data.mode()) > 0 else 'N/A':.4f}")
    
    print(f"\n  Medidas de Dispersión:")
    print(f"    • Desviación Estándar: {data.std():.4f}")
    print(f"    • Varianza: {data.var():.4f}")
    print(f"    • Rango: {data.max() - data.min():.4f}")
    print(f"    • Rango Intercuartílico (IQR): {data.quantile(0.75) - data.quantile(0.25):.4f}")
    
    print(f"\n  Cuartiles:")
    print(f"    • Q1 (25%): {data.quantile(0.25):.4f}")
    print(f"    • Q2 (50%): {data.quantile(0.50):.4f}")
    print(f"    • Q3 (75%): {data.quantile(0.75):.4f}")
    print(f"    • Q4 (100%): {data.max():.4f}")
    
    print(f"\n  Extremos:")
    print(f"    • Mínimo: {data.min():.4f}")
    print(f"    • Máximo: {data.max():.4f}")
    print(f"    • Percentil 5: {data.quantile(0.05):.4f}")
    print(f"    • Percentil 95: {data.quantile(0.95):.4f}")

print("\n" + "="*100)

In [ ]:
# Tabla resumen con describe
print("\n📊 Resumen Estadístico (Pandas describe):")
print(satisfaction_data.describe().T)

## 4. Análisis de Distribución

In [ ]:
print("📊 ANÁLISIS DE DISTRIBUCIÓN\n")
print("="*100)

for col in satisfaction_cols:
    print(f"\n🔹 {col.upper()}")
    print("-" * 100)
    
    data = panel_maestro[col].dropna()
    
    # Sesgo y Curtosis
    skewness = stats.skew(data)
    kurtosis = stats.kurtosis(data)
    
    print(f"  Asimetría (Skewness): {skewness:.4f}")
    if abs(skewness) < 0.5:
        print(f"    ➜ Distribución aproximadamente simétrica")
    elif skewness > 0:
        print(f"    ➜ Distribución sesgada a la DERECHA (cola derecha larga)")
    else:
        print(f"    ➜ Distribución sesgada a la IZQUIERDA (cola izquierda larga)")
    
    print(f"\n  Curtosis (Kurtosis): {kurtosis:.4f}")
    if abs(kurtosis) < 0.5:
        print(f"    ➜ Curtosis normal (mesocúrtica)")
    elif kurtosis > 0:
        print(f"    ➜ Distribución leptocúrtica (colas pesadas, picos altos)")
    else:
        print(f"    ➜ Distribución platicúrtica (colas ligeras, picos bajos)")
    
    # Test de normalidad (Shapiro-Wilk)
    if len(data) <= 5000:
        stat_shapiro, p_shapiro = stats.shapiro(data)
        print(f"\n  Test de Normalidad (Shapiro-Wilk):")
        print(f"    • Estadístico: {stat_shapiro:.4f}")
        print(f"    • p-valor: {p_shapiro:.6f}")
        if p_shapiro < 0.05:
            print(f"    ➜ ❌ Los datos NO siguen una distribución normal (p < 0.05)")
        else:
            print(f"    ➜ ✅ Los datos SÍ siguen una distribución normal (p ≥ 0.05)")
    
    # Test de Kolmogorov-Smirnov
    stat_ks, p_ks = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f"\n  Test de Kolmogorov-Smirnov:")
    print(f"    • Estadístico: {stat_ks:.4f}")
    print(f"    • p-valor: {p_ks:.6f}")
    if p_ks < 0.05:
        print(f"    ➜ ❌ Distribución rechazada como normal (p < 0.05)")
    else:
        print(f"    ➜ ✅ Distribución consistente con distribución normal (p ≥ 0.05)")

print("\n" + "="*100)

## 5. Visualización de Métricas de Satisfacción

In [ ]:
# Histogramas y Curvas de Densidad
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Histogramas y Densidad de Distribución - Métricas de Satisfacción', fontsize=16, fontweight='bold')

for idx, col in enumerate(satisfaction_cols):
    ax = axes[idx // 2, idx % 2]
    data = panel_maestro[col].dropna()
    
    # Histograma
    ax.hist(data, bins=30, alpha=0.7, color='steelblue', edgecolor='black', density=True, label='Histograma')
    
    # Curva de densidad
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(data)
    x_range = np.linspace(data.min(), data.max(), 200)
    ax.plot(x_range, kde(x_range), 'r-', linewidth=2, label='Densidad KDE')
    
    # Línea de media
    ax.axvline(data.mean(), color='green', linestyle='--', linewidth=2, label=f'Media: {data.mean():.2f}')
    ax.axvline(data.median(), color='orange', linestyle='--', linewidth=2, label=f'Mediana: {data.median():.2f}')
    
    ax.set_title(f'{col}\n(n={len(data)}, valores faltantes={panel_maestro[col].isna().sum()})', fontsize=12, fontweight='bold')
    ax.set_xlabel('Valor', fontsize=10)
    ax.set_ylabel('Densidad', fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('01_histogramas_densidad_satisfaccion.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado: 01_histogramas_densidad_satisfaccion.png")

In [ ]:
# Box Plots
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Box Plots - Detección de Outliers en Métricas de Satisfacción', fontsize=16, fontweight='bold')

for idx, col in enumerate(satisfaction_cols):
    ax = axes[idx // 2, idx % 2]
    data = panel_maestro[col].dropna()
    
    bp = ax.boxplot(data, vert=True, patch_artist=True, widths=0.5,
                    boxprops=dict(facecolor='lightblue', alpha=0.7),
                    medianprops=dict(color='red', linewidth=2),
                    whiskerprops=dict(color='black', linewidth=1.5),
                    capprops=dict(color='black', linewidth=1.5))
    
    # Calcular IQR y outliers
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    
    # Mostrar outliers
    ax.scatter([1]*len(outliers), outliers, color='red', s=100, zorder=3, alpha=0.6, label=f'Outliers (n={len(outliers)})')
    
    ax.set_title(f'{col}\nQ1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Valor', fontsize=10)
    ax.set_xticklabels([col])
    ax.grid(True, alpha=0.3, axis='y')
    ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('02_boxplots_satisfaccion.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado: 02_boxplots_satisfaccion.png")

In [ ]:
# Violin Plots
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Violin Plots - Distribución Detallada de Satisfacción', fontsize=16, fontweight='bold')

for idx, col in enumerate(satisfaction_cols):
    ax = axes[idx // 2, idx % 2]
    data = panel_maestro[col].dropna()
    
    parts = ax.violinplot([data], positions=[1], widths=0.7, showmeans=True, showmedians=True)
    
    for pc in parts['bodies']:
        pc.set_facecolor('steelblue')
        pc.set_alpha(0.7)
    
    ax.set_title(f'{col}\nMedia={data.mean():.2f}, Std={data.std():.2f}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Valor', fontsize=10)
    ax.set_xticklabels([col])
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('03_violinplots_satisfaccion.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado: 03_violinplots_satisfaccion.png")

In [ ]:
# Q-Q Plots para verificar normalidad
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Q-Q Plots - Verificación de Normalidad de Satisfacción', fontsize=16, fontweight='bold')

for idx, col in enumerate(satisfaction_cols):
    ax = axes[idx // 2, idx % 2]
    data = panel_maestro[col].dropna()
    
    stats.probplot(data, dist="norm", plot=ax)
    ax.set_title(f'{col}\nQ-Q Plot (Normal)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('04_qqplots_satisfaccion.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado: 04_qqplots_satisfaccion.png")

In [ ]:
# Comparación: Satisfacción de Alumnos vs Profesores
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Comparación: Satisfacción Alumnos vs Profesores', fontsize=16, fontweight='bold')

# Scatter plot
ax1 = axes[0]
valid_data = panel_maestro[['satisfaccion_alumnos', 'satisfaccion_profesores']].dropna()
ax1.scatter(valid_data['satisfaccion_alumnos'], valid_data['satisfaccion_profesores'], 
           alpha=0.6, s=50, color='steelblue', edgecolor='black', linewidth=0.5)
ax1.plot([0, 10], [0, 10], 'r--', linewidth=2, label='Línea de igualdad')
ax1.set_xlabel('Satisfacción Alumnos', fontsize=12, fontweight='bold')
ax1.set_ylabel('Satisfacción Profesores', fontsize=12, fontweight='bold')
ax1.set_title('Scatter Plot: Satisfacción Alumnos vs Profesores', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)

# Diferencia
ax2 = axes[1]
ax2.hist(valid_data['satisfaccion_profesores'] - valid_data['satisfaccion_alumnos'], 
        bins=30, alpha=0.7, color='coral', edgecolor='black')
ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='Diferencia = 0')
mean_diff = (valid_data['satisfaccion_profesores'] - valid_data['satisfaccion_alumnos']).mean()
ax2.axvline(mean_diff, color='green', linestyle='--', linewidth=2, label=f'Media diferencia: {mean_diff:.2f}')
ax2.set_xlabel('Diferencia (Profesores - Alumnos)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax2.set_title('Distribución de Diferencias: Satisfacción Profesores - Alumnos', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.savefig('05_comparacion_alumnos_profesores.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado: 05_comparacion_alumnos_profesores.png")

## 6. Detección de Outliers

In [ ]:
print("🔍 ANÁLISIS DE OUTLIERS\n")
print("="*100)

for col in satisfaction_cols:
    print(f"\n🔹 {col.upper()}")
    print("-" * 100)
    
    data = panel_maestro[col].dropna()
    
    # Método IQR (Interquartile Range)
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound_iqr = Q1 - 1.5 * IQR
    upper_bound_iqr = Q3 + 1.5 * IQR
    
    outliers_iqr = data[(data < lower_bound_iqr) | (data > upper_bound_iqr)]
    
    print(f"  Método IQR (Rango Intercuartílico):")
    print(f"    • Q1 = {Q1:.4f}, Q3 = {Q3:.4f}, IQR = {IQR:.4f}")
    print(f"    • Límite inferior: {lower_bound_iqr:.4f}")
    print(f"    • Límite superior: {upper_bound_iqr:.4f}")
    print(f"    • Outliers detectados: {len(outliers_iqr)} ({100*len(outliers_iqr)/len(data):.2f}%)")
    
    if len(outliers_iqr) > 0:
        print(f"    • Valores de outliers: {sorted(outliers_iqr.values)}")
    
    # Método Z-score
    z_scores = np.abs(stats.zscore(data))
    outliers_z = data[z_scores > 3]
    
    print(f"\n  Método Z-score (|Z| > 3):")
    print(f"    • Outliers detectados: {len(outliers_z)} ({100*len(outliers_z)/len(data):.2f}%)")
    
    if len(outliers_z) > 0:
        print(f"    • Valores de outliers: {sorted(outliers_z.values)}")
    
    # Método Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    outliers_if = iso_forest.fit_predict(data.values.reshape(-1, 1)) == -1
    
    print(f"\n  Método Isolation Forest:")
    print(f"    • Outliers detectados: {outliers_if.sum()} ({100*outliers_if.sum()/len(data):.2f}%)")

print("\n" + "="*100)

## 7. Evaluación de Calidad de Datos

In [ ]:
print("📋 EVALUACIÓN DE CALIDAD DE DATOS\n")
print("="*100)

print("\n🔍 VALORES FALTANTES:")
print("-" * 100)
missing_summary = pd.DataFrame({
    'Variable': satisfaction_cols,
    'Faltantes': [panel_maestro[col].isna().sum() for col in satisfaction_cols],
    'Porcentaje': [f"{100*panel_maestro[col].isna().sum()/len(panel_maestro):.2f}%" for col in satisfaction_cols],
    'Válidos': [panel_maestro[col].notna().sum() for col in satisfaction_cols]
})
print(missing_summary.to_string(index=False))

print("\n\n🔍 DUPLICADOS:")
print("-" * 100)
# Buscar filas completamente duplicadas
completely_duplicated = panel_maestro[satisfaction_cols].duplicated(keep=False).sum()
print(f"  • Filas completamente duplicadas en satisfacción: {completely_duplicated}")

# Buscar duplicados parciales
for col in satisfaction_cols:
    duplicated_count = panel_maestro[col].duplicated(keep=False).sum()
    print(f"  • Valores duplicados en {col}: {duplicated_count}")

print("\n\n🔍 CONSISTENCIA DE DATOS:")
print("-" * 100)
# Verificar que satisfaccion_promedio = (satisfaccion_alumnos + satisfaccion_profesores) / 2
valid_data_consistency = panel_maestro[['satisfaccion_alumnos', 'satisfaccion_profesores', 'satisfaccion_promedio']].dropna()
calculated_promedio = (valid_data_consistency['satisfaccion_alumnos'] + valid_data_consistency['satisfaccion_profesores']) / 2
consistency_check = np.isclose(valid_data_consistency['satisfaccion_promedio'], calculated_promedio, atol=0.01)
print(f"  • Registros consistentes (promedio = (alumnos + profesores) / 2): {consistency_check.sum()}/{len(consistency_check)}")
print(f"  • Registros inconsistentes: {(~consistency_check).sum()}")

# Verificar que diferencia_satis = satisfaccion_profesores - satisfaccion_alumnos
valid_data_diff = panel_maestro[['satisfaccion_alumnos', 'satisfaccion_profesores', 'diferencia_satis']].dropna()
calculated_diff = valid_data_diff['satisfaccion_profesores'] - valid_data_diff['satisfaccion_alumnos']
diff_check = np.isclose(valid_data_diff['diferencia_satis'], calculated_diff, atol=0.01)
print(f"  • Registros consistentes (diferencia = profesores - alumnos): {diff_check.sum()}/{len(diff_check)}")
print(f"  • Registros inconsistentes: {(~diff_check).sum()}")

print("\n\n🔍 RANGO DE VALORES:")
print("-" * 100)
for col in satisfaction_cols:
    data = panel_maestro[col].dropna()
    print(f"  • {col}:")
    print(f"    - Mínimo: {data.min():.4f} (esperado: >= 0)")
    print(f"    - Máximo: {data.max():.4f} (esperado: <= 10)")
    out_of_range = ((data < 0) | (data > 10)).sum()
    print(f"    - Valores fuera de rango [0-10]: {out_of_range}")

print("\n" + "="*100)

## 8. Resumen Ejecutivo

In [ ]:
print("\n" + "="*100)
print("📊 RESUMEN EJECUTIVO - ANÁLISIS UNIVARIANTE DE SATISFACCIÓN")
print("="*100)

print("\n🎯 HALLAZGOS PRINCIPALES:\n")

# 1. Nivel general de satisfacción
print("1️⃣ NIVEL GENERAL DE SATISFACCIÓN:")
satis_alumnos = panel_maestro['satisfaccion_alumnos'].mean()
satis_profesores = panel_maestro['satisfaccion_profesores'].mean()
satis_promedio = panel_maestro['satisfaccion_promedio'].mean()
print(f"   • Satisfacción de alumnos: {satis_alumnos:.2f}/10")
print(f"   • Satisfacción de profesores: {satis_profesores:.2f}/10")
print(f"   • Satisfacción promedio: {satis_promedio:.2f}/10")
print(f"   ➜ Los profesores reportan mayor satisfacción que los alumnos (+{satis_profesores-satis_alumnos:.2f} puntos)")

# 2. Distribución
print("\n2️⃣ CARACTERÍSTICAS DE DISTRIBUCIÓN:")
for col in ['satisfaccion_alumnos', 'satisfaccion_profesores']:
    data = panel_maestro[col].dropna()
    skewness = stats.skew(data)
    print(f"   • {col}:")
    print(f"     - Asimetría: {skewness:.4f} {'(izquierda)' if skewness < -0.5 else '(centro)' if abs(skewness) < 0.5 else '(derecha)'}")
    print(f"     - Desv. Estándar: {data.std():.4f}")

# 3. Calidad de datos
print("\n3️⃣ CALIDAD DE DATOS:")
missing_pct_alumnos = 100*panel_maestro['satisfaccion_alumnos'].isna().sum()/len(panel_maestro)
missing_pct_profesores = 100*panel_maestro['satisfaccion_profesores'].isna().sum()/len(panel_maestro)
print(f"   • Valores faltantes - Alumnos: {missing_pct_alumnos:.2f}%")
print(f"   • Valores faltantes - Profesores: {missing_pct_profesores:.2f}%")
print(f"   • Integridad: ✅ ACEPTABLE (< 5% de faltantes)")

# 4. Outliers
print("\n4️⃣ OUTLIERS:")
Q1_alumnos = panel_maestro['satisfaccion_alumnos'].quantile(0.25)
Q3_alumnos = panel_maestro['satisfaccion_alumnos'].quantile(0.75)
IQR_alumnos = Q3_alumnos - Q1_alumnos
outliers_alumnos = panel_maestro['satisfaccion_alumnos'][
    (panel_maestro['satisfaccion_alumnos'] < Q1_alumnos - 1.5*IQR_alumnos) | 
    (panel_maestro['satisfaccion_alumnos'] > Q3_alumnos + 1.5*IQR_alumnos)
]
print(f"   • Outliers en satisfacción alumnos: {len(outliers_alumnos)} ({100*len(outliers_alumnos)/panel_maestro['satisfaccion_alumnos'].notna().sum():.2f}%)")
print(f"   • Implicación: Algunos programas tienen satisfacción significativamente más baja")

# 5. Normalidad
print("\n5️⃣ NORMALIDAD DE DISTRIBUCIÓN:")
data_alumnos = panel_maestro['satisfaccion_alumnos'].dropna()
stat_shapiro, p_shapiro = stats.shapiro(data_alumnos)
print(f"   • Test Shapiro-Wilk (alumnos): p-valor = {p_shapiro:.6f}")
print(f"   • Conclusión: {'❌ NO normal' if p_shapiro < 0.05 else '✅ Normal'} (p < 0.05)")
print(f"   • Implicación: Se recomienda usar métodos no-paramétricos para inferencia")

print("\n" + "="*100)
print("✅ ANÁLISIS COMPLETADO")
print("="*100)